네이버 플레이스 리뷰 크롤링

In [ ]:
# Chrome 설치
!wget -q -O chrome.deb https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i chrome.deb || apt-get -f install -y

# Chromedriver 버전에 맞는 드라이버 설치 (현재 Chrome 버전 138 기준)
!wget -q https://storage.googleapis.com/chrome-for-testing-public/138.0.7204.49/linux64/chromedriver-linux64.zip
!unzip -o chromedriver-linux64.zip
!mv chromedriver-linux64/chromedriver /usr/bin/chromedriver
!chmod +x /usr/bin/chromedriver

(Reading database ... 126464 files and directories currently installed.)
Preparing to unpack chrome.deb ...
Unpacking google-chrome-stable (138.0.7204.49-1) over (138.0.7204.49-1) ...
Setting up google-chrome-stable (138.0.7204.49-1) ...
Processing triggers for mailcap (3.70+nmu1ubuntu1) ...
Processing triggers for man-db (2.10.2-1) ...
Archive:  chromedriver-linux64.zip
  inflating: chromedriver-linux64/LICENSE.chromedriver  
  inflating: chromedriver-linux64/THIRD_PARTY_NOTICES.chromedriver  
  inflating: chromedriver-linux64/chromedriver  


In [ ]:
!pip install selenium==4.22.0

In [ ]:
!pip install webdriver-manager openpyxl lxml

In [ ]:
!chromedriver --version
!chromium-browser --version

ChromeDriver 138.0.7204.49 (d2b48fd5f7813ed477a2d68fa232b8178fa4fb1e-refs/branch-heads/7204@{#1814})
/bin/bash: line 1: chromium-browser: command not found


In [ ]:
import uuid
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# ✅ 충돌 방지용 임시 사용자 디렉토리 설정
user_data_dir = f"/tmp/chrome-user-data-{uuid.uuid4()}"
options.add_argument(f'--user-data-dir={user_data_dir}')

options.binary_location = "/usr/bin/google-chrome"  # 핵심 포인트

driver = webdriver.Chrome(
    service=Service("/usr/bin/chromedriver"),
    options=options
)
driver.get("https://www.google.com")
print("✅ Page Title:", driver.title)
driver.quit()

✅ Page Title: Google


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import re

# ✅ 1. 셀레니움 옵션 세팅
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("user-agent=Mozilla/5.0 (iPhone; CPU iPhone OS 15_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Mobile/15E148 Safari/604.1")

In [ ]:
branch_names = [
    # 강남역
    "인생네컷 강남역1호점", "포토이즘박스 강남 1호점", "포토이즘박스 강남 2호점", "포토시그니처 강남점", "그믐달셀프스튜디오 강남역점",
    "하루필름 강남역점", "인생네컷 강남역 1호점", "돈룩업 강남", "비비드뮤지엄 강남역점", "포토이즘컬러드 강남점",
    # 잠실역
    "인생네컷 잠실", "필름한잔 롯데시네마 월드타워점", "포토이즘박스 롯데월드몰점", "포토이즘 박스 잠실역점",
    "포토이즘 박스 방이점", "하루필름 방이2호점", "돈룩업 송리단길", "포토이즘 박스 송리단길점",
    "포토이즘 컬러드 송리단길점", "비비드뮤지엄 송리단길점", "포토시그니처 송리단길",
    # 건대입구역
    "포토이즘박스 건대점", "포토그레이 건대점", "포토이즘컬러드 건대점", "비비드뮤지엄 건대점",
    "인생네컷 화양점", "인생네컷 세종대점", "에스앤에스 건대점", "돈룩업 건대", "포토이즘 박스 세종대점",
    "포토그레이 건대로데오점",
    # 홍대입구역
    "인생네컷 홍대입구역점", "포토그레이 홍대입구점", "비비드뮤지엄 홍대점", "청춘필름", "포토그레이 홍대 2호점",
    "돈룩업 홍대", "홍대네컷 홍대 2호점", "홍대네컷 홍대 3호점", "포토이즘컬러드 홍대점", "포토이즘 박스 홍대 어울림점"
]

In [ ]:
place_ids = [
    # 강남역
    "1025579134", "1062111114", "1880887682", "1889937968", "1946793246",
    "1738909007", "1279997652", "1558792785",
    # 잠실역
    "1090885328", "1698168444", "1759931381", "1150873745",
    "1869076645", "1977219458", "1427481796", "1696772144",
    "1041709293", "1686459945", "1012788076",
    # 건대입구역
    "1855857891", "1310100020", "1571367995", "1307698735",
    "1653333342", "1569544690", "1336927858", "1743035411", "1897503475",
    "1425003189",
    # 홍대입구역
    "1291592013", "1936794315", "1487670739", "1832677089", "1108180590",
    "1736590086", "1908669735", "1157637070", "1132795215", "1869488256",
    # 홍대 하루필름
    "1829431400"
]

In [ ]:
print(r.prettify())

<li class="place_apply_pui EjjAW">
 <div class="pui__q2fg8o pui__A7NplK">
  <a class="pui__sG3Q0N" data-pui-click-code="profile" href="https://m.place.naver.com/my/5d1592c533c14161c79e547e/review?v=2" role="button" target="_blank">
   <div class="pui__thumb" style="width:42px;height:42px">
    <img alt="프로필" class="pui__AnL5iU" height="38" src="https://search.pstatic.net/common/?autoRotate=true&amp;quality=95&amp;type=f48_48&amp;src=https%3A%2F%2Fphinf.pstatic.net%2Fcontact%2F20161227_233%2F1482833385437dPvV8_PNG%2Fimage.png" width="38"/>
    <div class="pui__8lw8E2">
     <img alt="프로필 띠" height="100%" src="https://myplace-phinf.pstatic.net/20230531_168/16855124866695Bca6_PNG/camp_profile_border.png" width="100%"/>
    </div>
   </div>
  </a>
  <a class="pui__hvyFHZ" data-pui-click-code="profile" href="https://m.place.naver.com/my/5d1592c533c14161c79e547e/review?v=2" role="button" target="_blank">
   <div class="pui__JiVbY3">
    <span class="pui__uslU0d">
     <span class="pui__NMi-D

In [ ]:
import time, uuid, datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from openpyxl import Workbook
from selenium.webdriver.common.action_chains import ActionChains

# WebDriver
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("window-size=390x844")  # 모바일 크기(플레이스는 모바일에서만 지원됨)
options.add_argument("user-agent=Mozilla/5.0 (iPhone; CPU iPhone OS 15_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Mobile/15E148 Safari/604.1")
options.add_argument(f"--user-data-dir=/tmp/chrome-user-data-{uuid.uuid4()}")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 엑셀 파일 생성
now = datetime.datetime.now()
wb = Workbook()
sheet = wb.active
sheet.title = "Reviews"
sheet.append(['Place ID', 'Place Name', 'Nickname', 'Content', 'Date', 'Revisit'])

# 각 place_ids에 대해 반복 수집
for pid in place_ids:
    try:
        url = f"https://m.place.naver.com/place/{pid}/review/visitor"
        driver.get(url)

        # 모든 '더보기' 클릭 루프
        for _ in range(30):
            try:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(1)

                more_button = WebDriverWait(driver, 3).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.fvwqf'))
                )
                driver.execute_script("arguments[0].scrollIntoView(true);", more_button)
                driver.execute_script("arguments[0].click();", more_button)
                print("더보기 클릭")
                time.sleep(2)
            except Exception as e:
                break

        # 전체 HTML 파싱
        soup = BeautifulSoup(driver.page_source, 'lxml')
        reviews = soup.select('li')

        # ✅ 가게 이름 추출
        place_name_tag = soup.select_one('meta[property="og:title"]')
        place_name = place_name_tag['content'].strip() if place_name_tag else 'Unknown'

        for r in reviews:
            nickname = r.select_one('div.pui__JiVbY3 > span.pui__uslU0d')
            content = r.select_one('div.pui__vn15t2 > a')
            date = r.select_one('div.pui__QKE5Pr > span.pui__gfuUIT > time')
            revisit_span = r.select('div.pui__QKE5Pr > span.pui__gfuUIT')

            nickname = nickname.text if nickname else ''
            content = content.get_text(strip=True) if content else ''
            date = date.text if date else ''
            revisit = revisit_span[1].text if len(revisit_span) > 1 else ''

            sheet.append([pid, place_name, nickname, content, date, revisit])

        print(f"✅ {pid} 완료 (리뷰 {len(reviews)}건)")

    except Exception as e:
        print(f"❌ {pid} 실패: {e}")
        continue

filename = f"naver_place_reviews_{now.strftime('%Y%m%d_%H%M%S')}.xlsx"
wb.save(filename)
print(f"📁 저장 완료: {filename}")

✅ 1025579134 완료 (리뷰 7건)
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 1062111114 완료 (리뷰 86건)
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 1880887682 완료 (리뷰 161건)
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 1889937968 완료 (리뷰 47건)
✅ 더보기 클릭
✅ 1946793246 완료 (리뷰 18건)
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 1738909007 완료 (리뷰 62건)
✅ 1279997652 완료 (리뷰 16건)
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 1558792785 완료 (리뷰 166건)
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 1090885328 완료 (리뷰 326건)
✅ 더보기 클릭
✅ 더보기 클릭
✅ 1698168444 완료 (리뷰 38건)
✅ 1759931381 완료 (리뷰 6건)
✅ 1150873745 완료 (리뷰 3건)
✅ 18690

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from openpyxl import Workbook
import time, uuid
from webdriver_manager.chrome import ChromeDriverManager

# Excel 파일 생성
wb = Workbook()
ws = wb.active
ws.title = "naver_blog_reviews"
ws.append(['Place ID', 'Place Name', 'Contents', 'Blog Link'])

# 반복 수집 시작
for place_id in place_ids:
    print(f"📍 Processing Place ID: {place_id}")
    url = f'https://m.place.naver.com/place/{place_id}/review/ugc?type=list'

    # 모바일 환경 설정
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("window-size=390x844")
    options.add_argument("user-agent=Mozilla/5.0 (iPhone; CPU iPhone OS 15_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Mobile/15E148 Safari/604.1")
    options.add_argument(f"--user-data-dir=/tmp/chrome-user-data-{uuid.uuid4()}")

    # 드라이버 실행 및 접속
    try:
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        driver.get(url)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "ul"))
        )

        # 모든 '더보기' 클릭 루프
        for _ in range(30):
            try:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(1)

                more_button = WebDriverWait(driver, 3).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.fvwqf'))
                )
                driver.execute_script("arguments[0].scrollIntoView(true);", more_button)
                driver.execute_script("arguments[0].click();", more_button)
                print("더보기 클릭")
                time.sleep(2)
            except Exception as e:
                break

        # 리뷰 본문 추출
        review_elements = driver.find_elements(By.CSS_SELECTOR, 'div.pui__vn15t2 > span')
        blog_links = driver.find_elements(By.CSS_SELECTOR, 'li.EblIP > a')

        # 가게 이름 추출
        place_name_tag = soup.select_one('meta[property="og:title"]')
        place_name = place_name_tag['content'].strip() if place_name_tag else 'Unknown'

        # 개수 동일한지 확인 (혹시 다르면 짧은 쪽 기준)
        for tag, link in zip(review_elements, blog_links):
            content = tag.text.strip()
            href = link.get_attribute("href")
            ws.append([place_id, place_name, content, href])

        print(f"{len(review_elements)}건 리뷰 추출됨")

        driver.quit()

    except Exception as e:
        print(f"실패: {place_id} / {e}")
        driver.quit()

# ✅ 엑셀 저장
save_name = 'naver_blog_reviews.xlsx'
wb.save(save_name)
print(f"💾 저장 완료: {save_name}")

📍 Processing Place ID: 1025579134
✅ 더보기 클릭
✅ 17건 리뷰 추출됨
📍 Processing Place ID: 1062111114
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 38건 리뷰 추출됨
📍 Processing Place ID: 1880887682
✅ 더보기 클릭
✅ 더보기 클릭
✅ 21건 리뷰 추출됨
📍 Processing Place ID: 1889937968
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 74건 리뷰 추출됨
📍 Processing Place ID: 1946793246
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 34건 리뷰 추출됨
📍 Processing Place ID: 1738909007
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 65건 리뷰 추출됨
📍 Processing Place ID: 1279997652
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 더보기 클릭
✅ 67건 리뷰 추출됨
📍 Processing Place ID: 1558792785
✅ 더보기 클릭
✅ 더보기 클릭
✅ 28건 리뷰 추출됨
📍 Processing Place ID: 1090885328
✅ 더보기 클릭
✅ 16건 리뷰 추출됨
📍 Processing Place ID: 1698168444
✅ 더보기 클릭
✅ 더보기 클릭
✅ 22건 리뷰 추출됨
📍 Processing Place ID: 1759931381
✅ 1건 리뷰 추출됨
📍 Processing Place ID: 1150873745
✅ 3건 리뷰 추출됨
📍 Processing Place ID: 1869076645
✅ 1건 리뷰 추출됨
📍 Processing Place ID: 1977219458
✅ 더보기 클릭
✅ 0건 리뷰 추출됨
📍 Processing Place ID: 1427481796
✅ 더보기 